# Bing search result to sentence label

## steps:

### 1- Getting Bing search result from search page link

### 2- Getting HTML for each search result from link

### 3- Getting article text from each HTML

### 4- Passing each sentence to Fine Bert and getting lable






## 7/4/2024
### Tina Nosrati

In [ ]:
#Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Packages
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import nltk
from nltk.tokenize import sent_tokenize
import re
import json
import time


In [ ]:

def bing_df(url, num_pages=5):
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Windows; Windows NT 10.2; Win64; x64; en-US) AppleWebKit/534.24 (KHTML, like Gecko) Chrome/50.0.2945.140 Safari/600.4 Edge/9.65857",
        "Mozilla/5.0 (Linux i654 x86_64; en-US) Gecko/20100101 Firefox/66.3",
        "Mozilla/5.0 (Windows; Windows NT 10.5; WOW64) AppleWebKit/533.12 (KHTML, like Gecko) Chrome/49.0.1035.350 Safari/600.6 Edge/11.33481",
        "Mozilla/5.0 (Linux; U; Linux i553 x86_64; en-US) Gecko/20130401 Firefox/67.7",
        "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 7_8_3; en-US) AppleWebKit/602.31 (KHTML, like Gecko) Chrome/54.0.1227.181 Safari/600",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_3; like Mac OS X) AppleWebKit/603.45 (KHTML, like Gecko) Chrome/50.0.2138.229 Mobile Safari/533.2",
        "Mozilla/5.0 (compatible; MSIE 11.0; Windows; U; Windows NT 6.2;; en-US Trident/7.0)",
        "Mozilla/5.0 (Windows NT 10.2; Win64; x64) AppleWebKit/534.30 (KHTML, like Gecko) Chrome/49.0.2187.339 Safari/600.8 Edge/11.98308",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 11_1_9; like Mac OS X) AppleWebKit/535.10 (KHTML, like Gecko) Chrome/48.0.2178.202 Mobile Safari/601.2",
        "Mozilla/5.0 (Linux; U; Linux x86_64) AppleWebKit/535.15 (KHTML, like Gecko) Chrome/48.0.1025.180 Safari/602"
    ]

    search_results = []

    for page in range(num_pages):
        headers = {'User-Agent': user_agents[page % len(user_agents)]}
        paginated_url = f"{url}&first={page * 10 + 1}"
        response = requests.get(paginated_url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            result_containers = soup.find_all('li', {'class': 'b_algo'})

            for result in result_containers:
                title_tag = result.find('h2')
                title = title_tag.text if title_tag else ''

                link_tag = result.find('a')
                link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else ''

                description_tag = result.find('p')
                description = description_tag.text if description_tag else ''

                displayed_link_tag = result.find('cite')
                displayed_link = displayed_link_tag.text if displayed_link_tag else ''

                snippet_tag = result.find('div', {'class': 'b_caption'})
                snippet = snippet_tag.text if snippet_tag else ''

                additional_info_tag = result.find('div', {'class': 'b_attribution'})
                additional_info = additional_info_tag.text if additional_info_tag else ''

                # Extracting the publication date if available
                publish_time_tag = result.find('span', {'class': 'news_dt'})
                publish_time = publish_time_tag.text if publish_time_tag else ''

                search_results.append({
                    'Title': title,
                    'Link': link,
                    'Description': description,
                    'Displayed Link': displayed_link,
                    'Snippet': snippet,
                    'Additional Info': additional_info,
                    'Publish Time': publish_time
                })
        else:
            print(f"Failed to retrieve page {page + 1}. Status code: {response.status_code}")
            break

        # Sleep for 5 seconds
        time.sleep(5)

    return pd.DataFrame(search_results)


In [ ]:
def fetch_html(url):
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        #print("flag")
        return response.text
    except requests.exceptions.RequestException as e:
        return None

In [ ]:
def strip_html_tags(html_content):
    if html_content is None:
        return ''
    soup = BeautifulSoup(html_content, 'html.parser')

    text = soup.get_text()

    return text

In [ ]:
def replace_multiple_newlines(text):
    if pd.isna(text):
        return text
    return re.sub(r'\n+', '\n', text)


In [ ]:
def remove_pattern(text):
    pattern_one_word_per_line = r'(\b\w+\b)\n'
    pattern_two_words_per_line = r'(\b\w+\b\s+\b\w+\b)\n'
    pattern_email = r'\S+@\S+\.\S+'
    pattern_date = r'\b(?:\d{1,2}[\/\.-])?(?:\d{1,2}[\/\.-])?\d{2,4}\b'

    text = re.sub(pattern_email, '', text)
    text = re.sub(pattern_date, '', text)
    text = re.sub(pattern_two_words_per_line, '', text)
    text = re.sub(pattern_one_word_per_line, '', text)

    return text

In [ ]:
def apply_remove_pattern_to_column(df, column_name):
    df[column_name] = df[column_name].apply(remove_pattern)
    return df

In [ ]:
nltk.download('punkt')

def text_to_dict(text):
    sentences = sent_tokenize(text)
    sentences_dict = {sentence: None for sentence in sentences}
    return sentences_dict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def classify_sentences(dicts_column):
    classified_dicts = []
    for item in dicts_column:
        if isinstance(item, str):
            dict_item = json.loads(item.replace("'", "\""))
        elif isinstance(item, dict):
            dict_item = item
        else:
            continue
        for sentence in dict_item.keys():
            truncated_sentence = sentence[:510]
            result = nlp(truncated_sentence)[0]
            dict_item[sentence] = result['label']
        classified_dicts.append(dict_item)
    return classified_dicts

In [ ]:
def save_dicts_to_json(df, folder_path):
  for index, row in df.iterrows():
    file_name = f"{index}.json"
    file_path = f"{folder_path}/{file_name}"
    with open(file_path, "w") as f:
      json.dump(row["dicts"], f)

In [ ]:
#Input
NJ_areas=pd.read_csv("/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/NJ_with_lat_long.csv")
NJ_areas.head()
NJ_areas['Bing Search Link'] = NJ_areas['Bing Search Link'].apply(lambda x: x.replace(" ", "+"))


In [ ]:
#for i in range(0,len(NJ_areas)):
  #folder_path = "content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Part 2 Areas/{}".format(NJ_areas.at[i,'Municipality'])
  #!mkdir -p "{folder_path}"



In [ ]:
NJ_areas

,Municipality,County,MRI Score,Latitude,Longitude,Bing Search Link
0,Aberdeen township,Monmouth,2.33,40.407114,-74.221629,https://www.bing.com/search?q=Aberdeen+townshi...
1,Absecon city,Atlantic,-2.94,39.436241,-74.517871,https://www.bing.com/search?q=Absecon+city+New...
2,Alexandria township,Hunterdon,3.95,40.591417,-75.012824,https://www.bing.com/search?q=Alexandria+towns...
3,Allamuchy township,Warren,4.03,40.921765,-74.810166,https://www.bing.com/search?q=Allamuchy+townsh...
4,Alloway township,Salem,0.83,39.560857,-75.362397,https://www.bing.com/search?q=Alloway+township...
...,...,...,...,...,...,...
348,Woodbury city,Gloucester,-6.38,39.845233,-75.147237,https://www.bing.com/search?q=Woodbury+city+Ne...
349,Woodland township,Burlington,-3.83,39.842075,-74.523058,https://www.bing.com/search?q=Woodland+townshi...
350,Woodstown borough,Salem,-1.92,39.651287,-75.329138,https://www.bing.com/search?q=Woodstown+boroug...
351,Woolwich township,Gloucester,2.89,39.741482,-75.348055,https://www.bing.com/search?q=Woolwich+townshi...


In [ ]:
for i in range(0,,len(NJ_areas)):
  folder_path = "/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Areas/{}".format(NJ_areas.at[i,'Municipality'])
  df = bing_df(NJ_areas.at[i,'Bing Search Link'], 5)
  df['Inner HTML'] = df['Displayed Link'].apply(fetch_html)
  df = df.dropna(subset=['Inner HTML'])
  df=df.reset_index()
  df=df.drop(columns=['index'])
  df['content'] = df['Inner HTML'].apply(strip_html_tags)
  df['content'] = df['content'].apply(replace_multiple_newlines)
  df = apply_remove_pattern_to_column(df, 'content')
  df['dicts'] = df['content'].apply(text_to_dict)
  finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg', num_labels=4)
  tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
  nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
  df['dicts'] = classify_sentences(df['dicts'])
  save_dicts_to_json(df, folder_path)